# 2 - Process spectral data

Loads adjusted data and formats for spectral analysis.

## Imports

Necessary modules for analysis.

In [1]:
# import modules

import xarray as xr
import numpy as np
import scipy.signal as sig
for i in range(2):
    %matplotlib notebook

In [2]:
# import data

adcp = 'Axis75'     # Slope(2013,2014,2017,2018), Axis75(2013,2014), Axis55(2017,2018)
year = 2013
ds_in = xr.open_dataset(f'../Data/data/adj/adj_{adcp}_{year}_0.nc')

n_seg = ds_in.n_seg
if n_seg > 1:
    ds = [ds_in]
    for i in range(n_seg):
        if i > 0:
            ds_temp = xr.open_dataset(f'../Data/data/adj/adj_{adcp}_{year}_{i}.nc')
            ds.append(ds_temp)
elif n_seg == 1:
    ds = [ds_in]
    
# print(ds)

In [3]:
# extract variables

t_stamp = ds[0].t_stamp
depth = ds[0].depth.values
d = ds[0].d
start_date = ds[0].start_date
end_date = ds[0].end_date

# get WKB scale for site specific depth range

scaling_array = np.load('../project/archive/N2/scaling_array.npy')
GM_depths = scaling_array[0]                                 # depths range from -4 to -924 metres
GM_scale = scaling_array[1]
int_scale = np.interp(depth,-GM_depths,GM_scale)

## Format spectrogram data
Removes the mean from each depth. Performs spectrogram process with optimised parameters for visual analysis of spectra.

In [4]:
# spectra data adjustments & Welch parameters

time_total = 0
for i in range(n_seg):
    time_total += len(ds[i].t_seg)
print('Total time length:',time_total)

# set Welch parameters for PSD

fs = 1.1111e-3                # 4 samples per HOUR, 1.11e-3 per SECOND
win = 'hann'                  # optimal window for averaging
nps = 1024                    # find optimal window for nperseg (1024 ~10 days)
overlap = nps // 2            # 50% overlap, default   

# set Welch parameters for spectrograms and depth-band plots

nps_s = 512                     # find optimal window for nperseg (1024 ~10 days)
overlap_s = nps_s // 2            # 50% overlap, default   

# remove short segments

t_short = []
for i in range(n_seg):
    if len(ds[i].t_seg) < nps:
        t_short.append(i)
for i in sorted(t_short, reverse=True):
    del ds[i]
n_seg = n_seg - len(t_short)

time_total = 0
for i in range(n_seg):
    time_total += len(ds[i].t_seg)
print(len(t_short),'segment(s) too short, new total time length:',time_total)

Total time length: 34366
0 segment(s) too short, new total time length: 34366


In [5]:
# remove mean at each depth

um,vm = [],[]
for i in range(n_seg):
    um_seg,vm_seg = [],[]
    uorig_temp = ds[i].uorig.values
    vorig_temp = ds[i].vorig.values
    for j in range(d):
        umtemp = np.copy(uorig_temp[:,j]) - np.nanmean(uorig_temp[:,j])
        vmtemp = np.copy(vorig_temp[:,j]) - np.nanmean(vorig_temp[:,j])
        um_seg.append(umtemp)
        vm_seg.append(vmtemp)
    um.append(um_seg)     # list[segment][depth][time]
    vm.append(vm_seg)     # 0 is upper depth index

In [6]:
# define cross-spectra function using spectrograms

def spectrocross(u,v,fs,window,nperseg,noverlap,nfft,detrend,return_onesided,scaling,axis,mode):
    freqs, time, Sxy = sig._spectral_helper(u, v, fs, window, nperseg,
                                            noverlap, nfft, detrend,
                                            return_onesided, scaling, axis,
                                            mode='psd')        # get this from scipy spectral.py in pkgs (add name to import list)
    return freqs, time, Sxy

In [7]:
# spectrograms at each depth (PSD at each depth in time-steps of nps)

spect,specf,spect_s,specf_s,Sxu,Sxv,cw,ccw,Sdu,Sdv = [],[],[],[],[],[],[],[],[],[]
cw_s, ccw_s = [],[]
for i in range(n_seg): 
    spect_temp,specf_temp,cw_temp,ccw_temp,spect_s_temp,specf_s_temp,Sxu_temp,Sxv_temp,Sdu_temp,Sdv_temp=[],[],[],[],[],[],[],[],[],[]
    cw_s_temp,ccw_s_temp = [],[]
    for z in range(d):
        # get spectrogram for PSD and rotary   
        u_f, u_t, u_Sxx = sig.spectrogram(um[i][z], fs=fs, window=win, \
                                                            nperseg = nps, noverlap = overlap, return_onesided=True)
        v_f, v_t, v_Sxx = sig.spectrogram(vm[i][z], fs=fs, window=win, \
                                                            nperseg = nps, noverlap = overlap, return_onesided=True)
        
        # get quadrature spectrogram and rotary
        uv_f,uv_t,uv_Cxy = spectrocross(vm[i][z],um[i][z],fs=fs,window=win,nperseg=nps,noverlap=overlap,nfft=None,detrend='constant',
                    return_onesided=True,scaling='density',axis=-1,mode='psd')     # cross spectrogram
        Sxyuv = uv_Cxy.imag                             # quadrature spectra is imag part of cross spectra
        Scw = ((u_Sxx + v_Sxx) - (2*Sxyuv)) / 2         # rotatory components
        Sccw = ((u_Sxx + v_Sxx) + (2*Sxyuv)) / 2 
        
        # get high-res for spectrogram and depth-band plots (finer resolution)
        u_f_s, u_t_s, u_Sxx_s = sig.spectrogram(um[i][z], fs=fs, window=win, \
                                                            nperseg = nps_s, noverlap = overlap_s, return_onesided=True)
        v_f_s, v_t_s, v_Sxx_s = sig.spectrogram(vm[i][z], fs=fs, window=win, \
                                                            nperseg = nps_s, noverlap = overlap_s, return_onesided=True)
        
        # get high-res quadrature spectrogram and rotary
        uv_f_s,uv_t_s,uv_Cxy_s = spectrocross(vm[i][z],um[i][z],fs=fs,window=win,nperseg=nps_s,noverlap=overlap_s,nfft=None,
                                              detrend='constant',return_onesided=True,scaling='density',
                                              axis=-1,mode='psd')     # cross spectrogram
        Sxyuv_s = uv_Cxy_s.imag                                 # quadrature spectra is imag part of cross spectra
        Scw_s = ((u_Sxx_s + v_Sxx_s) - (2*Sxyuv_s)) / 2         # rotatory components
        Sccw_s = ((u_Sxx_s + v_Sxx_s) + (2*Sxyuv_s)) / 2 
            
        # convert spectro_t to datetime for x-axis on plots for PSD
        spectro_t4 = u_t*fs
        spectro_time_len = len(spectro_t4)
        spectro_time_axis = np.zeros([spectro_time_len],dtype='datetime64[s]')
        for k in range(spectro_time_len):
            j = np.int(spectro_t4[k])
            spectro_time_axis[k] = ds[i].t_seg[j].values    
            
        # convert spectro_t to datetime for x-axis on plots for spectro/depth-band
        spectro_t4_s = u_t_s*fs
        spectro_time_len_s = len(spectro_t4_s)
        spectro_time_axis_s = np.zeros([spectro_time_len_s],dtype='datetime64[s]')
        for k in range(spectro_time_len_s):
            j = np.int(spectro_t4_s[k])
            spectro_time_axis_s[k] = ds[i].t_seg[j].values

        spect_temp.append(spectro_time_axis)
        specf_temp.append(u_f)
        cw_temp.append(Scw)
        ccw_temp.append(Sccw)
        cw_s_temp.append(Scw_s)
        ccw_s_temp.append(Sccw_s)
        spect_s_temp.append(spectro_time_axis_s)
        specf_s_temp.append(u_f_s)
        Sxu_temp.append(u_Sxx)         # non-whitened
        Sxv_temp.append(v_Sxx)
        Sdu_temp.append(u_Sxx_s)
        Sdv_temp.append(v_Sxx_s)
        
    spect.append(spect_temp[0])   # append arrays to lists (same times and frequencies at each depth)
    specf.append(specf_temp[0])
    spect_s.append(spect_s_temp[0])
    specf_s.append(specf_s_temp[0])
    Sxu.append(Sxu_temp)          # PSD
    Sxv.append(Sxv_temp)
    cw.append(cw_temp)            # rotary spectra
    ccw.append(ccw_temp)
    cw_s.append(cw_s_temp)
    ccw_s.append(ccw_s_temp)
    Sdu.append(Sdu_temp)          # non-whitened spectro/depth-band
    Sdv.append(Sdv_temp)          # list[array_segment][depth][frequency][time_segment]

## Save
Save key values and arrays to NetCDF format using xarray.

In [16]:
# save to .nc files

for i in range(n_seg):
    ds_out = xr.Dataset( 
        data_vars=dict(
            Sxxu=(['depth','specf','spect'], Sxu[i]),                   # depth-freq-time data for PSD at each depth
            Sxxv=(['depth','specf','spect'], Sxv[i]),
            cw=(['depth','specf','spect'], cw[i]),                      # depth-freq-time data for rotary at each depth
            ccw=(['depth','specf','spect'], ccw[i]),
            Sxxu_d=(['depth','specf_s','spect_s'], Sdu[i]),             # depth-freq-time data for depth-band/spectro at each depth
            Sxxv_d=(['depth','specf_s','spect_s'], Sdv[i]),
            cw_d=(['depth','specf_s','spect_s'], cw_s[i]),              # depth-freq-time data for rotary for depth-band/spectro
            ccw_d=(['depth','specf_s','spect_s'], ccw_s[i]),
            WKB_scale=(['depth'],int_scale),                            # WKB multiplier
        ),
        coords=dict(
            depth=depth,                               # depth values
            specf=specf[i],                            # PSD frequency bins
            specf_s=specf_s[i],                        # spectrogram frequency bins
            spect=spect[i],                            # PSD time bins
            spect_s=spect_s[i],                        # spectrogram time bins
        ),
        attrs=dict(
            description=f'Depth-freq-time data for {adcp} {t_stamp} segment {i} (from 0 to {n_seg - 1}).',
            n_seg=n_seg,                               # n_seg for major datagaps each year
            adcp=adcp,                                 # adcp
            t_stamp=t_stamp,                           # time stamp
            t=time_total,                              # length of initial time series
            d=d,                                       # depth range
            start_date=str(ds[i].t_seg[0].values),     # start date of initial time series segment
            end_date=str(ds[i].t_seg[-1].values),      # end date of initial time series segment
            fs=fs,                                     # sampling frequency
            window=win,                                # fft window
            nps=nps,                                   # fft segment length
            overlap=overlap,                           # fft window overlap
            nps_s=nps_s,                               # fft segment length for depth-band and spetro
            overlap_s=overlap_s,                       # fft window overlap for depth-band and spectro
        ),
    ) 

    ds_out.to_netcdf(f'../Data/data/spectra/spectra_{adcp}_{t_stamp}_{i}.nc')